In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import torch as t
import torch.nn.functional as F
from data import *
from train import *
from tune import *
from haikunator import Haikunator

In [ ]:
DATAROOT = Path.home() / "mldata"
RUNROOT = Path.home() / "mlruns" / "imdb"
t.set_printoptions(precision=8)

In [ ]:
vocab, _ = get_vocab(DATAROOT)

In [ ]:
len(vocab)

## Experiment 1
Lets learn the embeddings along with the rest of the NN. The NN architecture is very simple, a couple of fully connected layers with ReLU activation. After hyperparameter tuning the best hyper parameters are:
  * max_epochs: 9
  * learning_rate: 0.00158223651599205
  * max_seq_len: 100
  * batch_size': 8
  * embedding_dim: 25

Giving an accuracy of 73.79%

In [ ]:
from models import Simple

simple = Simple(vocab_size=len(vocab), max_seq_len=100, embedding_dim=25)
print(simple)

In [ ]:
# logger = pl.loggers.CSVLogger(save_dir=RUNROOT.parent, name=RUNROOT.name) # type: ignore
trainer = pl.Trainer(
    default_root_dir=RUNROOT,
    max_epochs=9,
    logger=True,
    num_sanity_val_steps=0,
    enable_progress_bar=True,
    # MPS does not work because aten::unique op is not implemented
    # accelerator='mps', 
    # devices=1
    # gpus=1 if t.cuda.is_available() else None
    accelerator="gpu" if t.cuda.is_available() else "cpu",
    devices=1
)

In [ ]:
dm = ImdbDataModule(
    dataroot=DATAROOT,
    vocab=vocab,
    max_seq_len=100,
    batch_size=8
)

model = LitImdb(
    model_factory=Simple,  # type: ignore
    vocab_size=len(vocab),
    max_seq_len=100,
    embedding_dim=25,
    learning_rate=0.00158223651599205
)

trainer.fit(model, dm)
trainer.validate(datamodule=dm)

Best Params: {'max_epochs': 9, 'max_seq_len': 500, 'batch_size': 64, 'embedding_dim': 200, 'learning_rate': 0.0009252345757324095}
Values: ({'accuracy': 0.6941666603088379}, {'accuracy': {'accuracy': 0.0}})